In [1]:
import os

with open(os.path.join('data', 'joke', 'data')) as f:
    lines = f.readlines()

for line in lines:
    if 'ID' in line: continue
    print(line)
    break

1,"What did the bartender say to the jumper cables? You better not try to start anything."



In [2]:
joke_id, joke = tuple(line.split(',"'))

In [3]:
from torch.utils.data import DataLoader, random_split
from transformers import AutoTokenizer

from nlp import datasets
import importlib
importlib.reload(datasets)

c:\Users\jnicolow\AppData\Local\anaconda3\envs\nlp-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'nlp.datasets' from 'c:\\Users\\jnicolow\\Documents\\courses\\fall2024\\ICS-661\\llm_finetuning\\nlp\\datasets.py'>

In [4]:
importlib.reload(datasets)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token}) # there is no default


batch_size = 2

dataset = datasets.JokesDataset(filepath=os.path.join('data', 'joke', 'data'), tokenizer=tokenizer)
train_ratio = 0.9
eval_ratio = 1 - train_ratio
total_size = len(dataset)
train_size = int(total_size * train_ratio)
eval_size = total_size - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)



print(f'there are {len(train_dataset)} training examples')
print(f'there are {len(eval_dataset)} test examples')

for batch in train_loader:
    print(tokenizer.decode(batch['input_ids'][0]))
    print(tokenizer.decode(batch['target_ids'][0]))
    break

there are 1459 training examples
there are 163 test examples
There are three types of people in this world. Those who can count and those who can't."
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|end

In [56]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # resize the model's token embeddings to accommodate the new padding token

Embedding(50257, 768)

In [57]:
import torch
from torch.optim import AdamW

device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu') # second gpu has more mem
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)  

def train_model(model, train_loader, optimizer, device, num_epochs=3):
    model.train()  
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:

            input_ids = batch['input_ids'].to(device)
            labels = input_ids.clone()  
            labels[labels == '[PAD]'] = -100  
            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss  
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')

def evaluate_model(model, eval_loader, device):
    model.eval() 
    total_loss = 0

    with torch.no_grad():
        for batch_index, batch in enumerate(eval_loader):
            input_ids = batch['input_ids'].to(device)
            labels = input_ids.clone()  

            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()


            if batch_index == 0:  
                for i in range(min(2, input_ids.size(0))):  
                    decoded_input = tokenizer.decode(input_ids[i], skip_special_tokens=True)
                    print(f'Example {i + 1}: {decoded_input}')
            

    avg_loss = total_loss / len(eval_loader)
    print(f'Evaluation Loss: {avg_loss:.4f}')


train_model(model, train_loader, optimizer, device, num_epochs=15)

evaluate_model(model, eval_loader, device)

# save model and tokenizer
model.save_pretrained('models/fine_tuned_gpt2')
tokenizer.save_pretrained('models/fine_tuned_gpt2')


Epoch 1/15, Loss: 0.8616
Epoch 2/15, Loss: 0.6898
Epoch 3/15, Loss: 0.6149
Epoch 4/15, Loss: 0.5517
Epoch 5/15, Loss: 0.4947
Epoch 6/15, Loss: 0.4368
Epoch 7/15, Loss: 0.3845
Epoch 8/15, Loss: 0.3376
Epoch 9/15, Loss: 0.2938
Epoch 10/15, Loss: 0.2537
Epoch 11/15, Loss: 0.2229
Epoch 12/15, Loss: 0.1957
Epoch 13/15, Loss: 0.1736
Epoch 14/15, Loss: 0.1540
Epoch 15/15, Loss: 0.1413
Example 1: Where do you go to weigh a pie? Somewhere over the rainbow weigh a pie. (sounds like way up high)"

Example 2: What is a rocket's favorite meal? Launch! Another one from my 9 year old."

Evaluation Loss: 1.1074


('models/fine_tuned_gpt2\\tokenizer_config.json',
 'models/fine_tuned_gpt2\\special_tokens_map.json',
 'models/fine_tuned_gpt2\\vocab.json',
 'models/fine_tuned_gpt2\\merges.txt',
 'models/fine_tuned_gpt2\\added_tokens.json',
 'models/fine_tuned_gpt2\\tokenizer.json')

In [14]:
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("models/fine_tuned_gpt2") # load tokenizer
model = AutoModelForCausalLM.from_pretrained('models/fine_tuned_gpt2') # load model
model.eval()  

def generate_random_words(word_list, num_words=3):
    return random.sample(word_list, num_words)

def generate_joke(starting_words, max_length=50):
    prompt = ' '.join(starting_words) 
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    
    
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

    generated_ids = output[0]  # get the first output (for a single sample)
    generated_ids = generated_ids.tolist()  # convert tensor to list
    
    
    joke = tokenizer.decode(generated_ids, skip_special_tokens=False)
    return joke


word_list = ['cat', 'dog', 'fish', 'funny', 'computer', 'banana', 'rocket', 'pizza', 'monkey', 'joke'] # random words are generated from this


random_words = generate_random_words(word_list)

# random_words = ['What do you']
joke = generate_joke(random_words)
print("Starting Words:", random_words)
print("Generated Joke:", joke)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Starting Words: ['cat', 'dog', 'banana']
Generated Joke: cat dog banana walk into a bar The bartender says, ""What's with the banana walk?"" And the banana walks away."
<|endoftext|>
